In [3]:
import json
import sys
sys.path.append("../src")

import ipywidgets as widgets
import numpy as np
from ipycanvas import Canvas
from ipywidgets import Image

from stringart.utils import string_chord, nail_point

In [31]:
# number of nails
NUM_NAILS=200
# fraction of arc between adjacent nails occupied by one nail
NAIL_FRAC=0.2
# compute radius (alt, specify directly)
NAIL_RADIUS= np.pi / NUM_NAILS * NAIL_FRAC

# canvas height/width
PIXELS = 800
# alpha color channel for each string
STRING_ALPHA=0.2
LINE_WIDTH=0.5

def pixel(x):
    """Convert (0, 1) to (0, PIXELS) range"""
    return x * PIXELS

canvas = Canvas(width=PIXELS, height=PIXELS)
canvas.line_width = LINE_WIDTH

with open("../src/cycle_final.json") as f:
    cycle = json.load(f)

def draw_strings(seq_idx):

    idxs = []

    canvas.stroke_circles(
        [PIXELS*nail_point(i, NUM_NAILS)[0] for i in range(NUM_NAILS)],
        [PIXELS*nail_point(i, NUM_NAILS)[1] for i in range(NUM_NAILS)],
        PIXELS*NAIL_RADIUS,
    )
    
    points = []
    alphas = []

    for i in range(seq_idx):
        m = cycle[i]
        n = cycle[i+1]
        mv, nv = string_chord(m, n, NUM_NAILS, NAIL_RADIUS)
        mx, my = mv
        nx, ny = nv
        points.append([[pixel(mx), PIXELS-pixel(my)], [pixel(nx), PIXELS-pixel(ny)]])
        alphas.append(STRING_ALPHA)

    canvas.stroke_styled_line_segments(
        np.array(points, dtype=int),
        np.array([[0, 0, 1]], dtype=int),
        np.array(alphas),
    )
    
    display(canvas)

image = Image.from_file("../src/grayscale.png")

image_toggle = widgets.ToggleButtons(
    options=['Strings', 'Strings & Image', 'Image'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

seq_range = widgets.IntSlider(value=10, min=10, max=len(cycle)-1, step=10, continuous_update=True)

def draw_canvas():
    
    canvas.clear()
    canvas.stroke_circle(pixel(0.5), pixel(0.5), PIXELS/2)
    if 'Image' in image_toggle.value:
        canvas.draw_image(image, 0, 0, PIXELS, PIXELS)
    if 'Strings' in image_toggle.value:
        draw_strings(seq_range.value)
        

display(image_toggle, seq_range)
draw_canvas()

def on_seq_value_change(change):
    draw_canvas()

def on_toggle_value_change(change):
    draw_canvas()
    

seq_range.observe(on_seq_value_change, names='value')
image_toggle.observe(on_toggle_value_change, names='value')

ToggleButtons(options=('Strings', 'Strings & Image', 'Image'), value='Strings')

IntSlider(value=10, max=2800, min=10, step=10)

Canvas(height=800, width=800)